# **PROYECTO INDIVIDUAL 1**
## Machine Learning Operations (MLOps)

## Índice de contenido:

1. [Ingesta de datos](#1-ingesta-de-datos)
2. [Tratamiento de datos](#2-tratamiento-de-datos)
3. [Feature Engineering](#3-feature-engineering)
4. [Funciones y disponibilización de datos](#4-desarrollo-de-funciones-y-disponibilización-de-datos-api)
5. [Deployment](#5-deployment)
6. [Análisis exploratorio (EDA)](#6-analisis-exploratorio)
7. [Modelamiento (Machine Learning Model Development)](#7-modelamiento)


# 1. **Ingesta de datos**
***

## 1.1 Importamos librerías

Importamos las librerias que vamos a usar durante el desarrollo del proyecto.

In [2]:
import pandas as pd 
import numpy as np
import json
import gzip
import ast

## 1.2 Leemos los datos

Utilizaremos 3 archivos: steam_games, user_reviews y users_items. 

In [3]:
ruta1 = '/Users/jpbertone/Documents/Henry Course/Labs/dataPI_MLOps/steam_games.json'
ruta2 = '/Users/jpbertone/Documents/Henry Course/Labs/dataPI_MLOps/user_reviews.json.gz'
ruta3 = '/Users/jpbertone/Documents/Henry Course/Labs/dataPI_MLOps/users_items.json.gz'

Descripción de los datasets:

1. **steam_games:** información relacionada a los juegos dentro de la plataforma Steam. Por ejemplo: Nombre del juego, género, fecha de lanzamiento, entre otras. 

2. **user_reviews:** información que detalla las reviews realizadas por los usuarios de la plataforma Steam.

3. **users_items:** información acerca de la actividad de los usuarios dentro de la plataforma Steam.

- ### 1.2.1 **steam_games**

Mediante el uso de Pandas, leemos el archivo 'steam_games' y lo asignamos a un nombre identificatorio.

In [4]:
df_games = pd.read_json(ruta1, lines=True)

In [5]:
df_games.tail(2)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,discount_price,specs,price,early_access,id,metascore,developer,user_id,steam_id,items,items_count
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,NaN,"[Single-player, Steam Achievements, Steam Cloud]",4.99,0.0,658870.0,None,"xropi,stev3ns",None,NaN,None,NaN
120444,None,None,Maze Run VR,None,http://store.steampowered.com/app/681550/Maze_...,None,"[Early Access, Adventure, Indie, Action, Simul...",http://steamcommunity.com/app/681550/reviews/?...,NaN,"[Single-player, Stats, Steam Leaderboards, HTC...",4.99,1.0,681550.0,None,None,None,NaN,None,NaN


- ### 1.2.2 **user_reviews**

Para leer los archivos user_reviews y users_items, utilizamos otro método.

A diferencia del dataset anterior el cual se encuentra formato **.json**, en este caso abrimos un archivo **.json** comprimido en formato **.gzip**

Se tomó la decisión de leerlo de otra manera dado que estos dos datasets presentan una estructura **.json** distinta aa acpetada por el comando pd.read_json. Esas diferencias son, en su gran mayoría, el uso de comillas simples en vez de dobles. 

Para lograr leer el archivo seguimos los siguientes pasos:

In [6]:
info = [] # Creamos una lista vacia donde guardaremos los registros del archivo .json

# Iteramos sobre los registros del archivo y con <ast.literal_eval> de forma segura una expresión
# literal que está representada como una cadena de caracteres.
for i in gzip.open(ruta2):
     info.append(ast.literal_eval(i.decode('utf-8')))
     
df_reviews = pd.DataFrame(info) 

In [7]:
df_reviews.head(5)

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


- ### 1.2.3 **users_items**

In [8]:
info = [] # Creamos una lista vacia donde guardaremos los registros del archivo .json

# Iteramos sobre los registros del archivo y con <ast.literal_eval> de forma segura una expresión
# literal que está representada como una cadena de caracteres.
for i in gzip.open(ruta3):
     info.append(ast.literal_eval(i.decode('utf-8')))
     
df_items = pd.DataFrame(info) 

In [9]:
df_items.head(5)

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


## 1.3 Creación de archivos

En este paso vamos a dejar una copia guardada de los dataframes creados en los pasos anteriores. Vamos a convertirlos a **parquet**

# 2. **Tratamiento de datos (ETL)**

***

## 2.1 Desanidar columnas

Luego de leer los datos, se observa que los tres dataframes presentan columnas con datos anidados.

Para ello, vamos a utilizar las funciones .explode y .json_normalize, con el objetivo de desanidar la informacion y convertir cada clave a columna.

Luego de desanidar, concatenamos las columnas del data frame df_explode con las columnas desanidadas. 

- ### 2.1.1  **df_games** 

    En este caso no desanidamos y eliminamos la columna items, items_count, user_id y steam_id. 

    Se decide eliminar estas columnas porque ya tenemos esta infomacion en el dataset df_items_full. 


In [10]:
df_games_full = df_games.copy()
df_games_full = df_games_full.drop(columns=['items','items_count', 'user_id', 'steam_id'],axis=1)

In [11]:
df_games_full.tail(2)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,discount_price,specs,price,early_access,id,metascore,developer
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,NaN,"[Single-player, Steam Achievements, Steam Cloud]",4.99,0.0,658870.0,None,"xropi,stev3ns"
120444,None,None,Maze Run VR,None,http://store.steampowered.com/app/681550/Maze_...,None,"[Early Access, Adventure, Indie, Action, Simul...",http://steamcommunity.com/app/681550/reviews/?...,NaN,"[Single-player, Stats, Steam Leaderboards, HTC...",4.99,1.0,681550.0,None,None


- ### 2.1.2  **df_reviews** 

    Desanidamos la informacion que se encuentra en la columna 'reviews'.

In [12]:
df_explode = df_reviews.explode('reviews')
df_explode.dropna(how='all',inplace=True)
df_explode.reset_index(inplace=True, drop=True)

reviews = pd.json_normalize(df_explode['reviews'].dropna())
reviews.reset_index(inplace=True, drop=True)

df_reviews_full = pd.concat([df_explode.drop(columns='reviews'), reviews], axis=1)

In [13]:
df_reviews_full.head(2)

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.


- ### 2.1.3  **df_items** 

    Desanidamos la informacion que se encuentra en la columna 'items'.

In [14]:
df_explode = df_items.explode('items').reset_index(drop=True)

items = pd.json_normalize(df_explode['items']).reset_index(drop=True)

df_items_full = pd.concat([df_explode.drop('items', axis=1), items], axis=1)

In [15]:
df_items_full.head(2)

,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,10,Counter-Strike,6.0,0.0
1,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,20,Team Fortress Classic,0.0,0.0


## 2.2 Duplicados, valores nulos e inconsistencias

- ### 2.2.1 **df_games_full**

In [16]:
df_games_full.shape

(120445, 15)

In [17]:
df_games_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   publisher       24083 non-null  object 
 1   genres          28852 non-null  object 
 2   app_name        32133 non-null  object 
 3   title           30085 non-null  object 
 4   url             32135 non-null  object 
 5   release_date    30068 non-null  object 
 6   tags            31972 non-null  object 
 7   reviews_url     32133 non-null  object 
 8   discount_price  225 non-null    float64
 9   specs           31465 non-null  object 
 10  price           30758 non-null  object 
 11  early_access    32135 non-null  float64
 12  id              32133 non-null  float64
 13  metascore       2677 non-null   object 
 14  developer       28836 non-null  object 
dtypes: float64(3), object(12)
memory usage: 13.8+ MB


Vamos a observar si la columna 'precio' tiene valores numericos o otro tipo de datos que deberían ser sustituidos. 

In [18]:
df_games_full['price'].unique()

array([None, 4.99, 'Free To Play', 'Free to Play', 0.99, 2.99, 3.99, 9.99,
       18.99, 29.99, 'Free', 10.99, 1.5899999999999999, 14.99, 1.99,
       59.99, 8.99, 6.99, 7.99, 39.99, 19.99, 7.49, 12.99, 5.99, 2.49,
       15.99, 1.25, 24.99, 17.99, 61.99, 3.49, 11.99, 13.99, 'Free Demo',
       'Play for Free!', 34.99, 74.76, 1.49, 32.99, 99.99, 14.95, 69.99,
       16.99, 79.99, 49.99, 5.0, 44.99, 13.98, 29.96, 119.99, 109.99,
       149.99, 771.71, 'Install Now', 21.99, 89.99,
       'Play WARMACHINE: Tactics Demo', 0.98, 139.92, 4.29, 64.99,
       'Free Mod', 54.99, 74.99, 'Install Theme', 0.89, 'Third-party',
       0.5, 'Play Now', 299.99, 1.29, 3.0, 15.0, 5.49, 23.99, 49.0, 20.99,
       10.93, 1.3900000000000001, 'Free HITMAN™ Holiday Pack', 36.99,
       4.49, 2.0, 4.0, 9.0, 234.99, 1.9500000000000002, 1.5, 199.0, 189.0,
       6.66, 27.99, 10.49, 129.99, 179.0, 26.99, 399.99, 31.99, 399.0,
       20.0, 40.0, 3.33, 199.99, 22.99, 320.0, 38.85, 71.7, 59.95, 995.0,
       27.49,

Efectivamente, y como se observa en la lista de arriba, podemos observar que varios valores no son numericos y en su gran mayoría representan juegos gratuitos o demos. 

Vamos a iterar la columna con el objetivo de reemplzar esos valores por **0**. 

Ademas, observamos que existe un juego que su precio base es de $499 ('Starting at $499.0). En este caso vamos a reemplazarlo por el valor **499**

In [19]:
for i in df_games_full['price']:
    # Si dentro de la columna existe algun valor de la lista, lo reemplaza por el valor numérico 0.
    if i in ['Free to Try', 
             'Free to Play', 
             'Free To Play', 
             'Free', 'Free Demo', 
             'Play for Free!',
             'Install Now', 
             'Play WARMACHINE: Tactics Demo', 
             'Free Mod', 
             'Install Theme', 
             'Third-party', 
             'Play Now',
             'Free HITMAN™ Holiday Pack', 
             'Play the Demo', 
             'Free Movie', 
             'Free to Use']:
        df_games_full.replace(i, 0, inplace = True)
    # Si encuentra el texto, lo reemplaza por el valor numérico 499.
    elif i in ['Starting at $499.00','Starting at $449.00']:
        df_games_full.replace(i, 499, inplace = True) 



In [20]:
df_games_full['price'].unique()

array([       nan, 4.9900e+00, 0.0000e+00, 9.9000e-01, 2.9900e+00,
       3.9900e+00, 9.9900e+00, 1.8990e+01, 2.9990e+01, 1.0990e+01,
       1.5900e+00, 1.4990e+01, 1.9900e+00, 5.9990e+01, 8.9900e+00,
       6.9900e+00, 7.9900e+00, 3.9990e+01, 1.9990e+01, 7.4900e+00,
       1.2990e+01, 5.9900e+00, 2.4900e+00, 1.5990e+01, 1.2500e+00,
       2.4990e+01, 1.7990e+01, 6.1990e+01, 3.4900e+00, 1.1990e+01,
       1.3990e+01, 3.4990e+01, 7.4760e+01, 1.4900e+00, 3.2990e+01,
       9.9990e+01, 1.4950e+01, 6.9990e+01, 1.6990e+01, 7.9990e+01,
       4.9990e+01, 5.0000e+00, 4.4990e+01, 1.3980e+01, 2.9960e+01,
       1.1999e+02, 1.0999e+02, 1.4999e+02, 7.7171e+02, 2.1990e+01,
       8.9990e+01, 9.8000e-01, 1.3992e+02, 4.2900e+00, 6.4990e+01,
       5.4990e+01, 7.4990e+01, 8.9000e-01, 5.0000e-01, 2.9999e+02,
       1.2900e+00, 3.0000e+00, 1.5000e+01, 5.4900e+00, 2.3990e+01,
       4.9000e+01, 2.0990e+01, 1.0930e+01, 1.3900e+00, 3.6990e+01,
       4.4900e+00, 2.0000e+00, 4.0000e+00, 9.0000e+00, 2.3499e

Ahora podemos observar que el esos valores ya no existen y todos los valores que tenemos en la columna prices son numericos. 

El siguiente paso es convertir la columna a tipo numerico **float**.

In [21]:
df_games_full['price'] = df_games_full['price'].astype(float)

In [22]:
df_games_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   publisher       24083 non-null  object 
 1   genres          28852 non-null  object 
 2   app_name        32133 non-null  object 
 3   title           30085 non-null  object 
 4   url             32135 non-null  object 
 5   release_date    30068 non-null  object 
 6   tags            31972 non-null  object 
 7   reviews_url     32133 non-null  object 
 8   discount_price  225 non-null    float64
 9   specs           31465 non-null  object 
 10  price           30758 non-null  float64
 11  early_access    32135 non-null  float64
 12  id              32133 non-null  float64
 13  metascore       2677 non-null   object 
 14  developer       28836 non-null  object 
dtypes: float64(4), object(11)
memory usage: 13.8+ MB


Como se observa en el resumen de arriba, el datasets presenta una gran cantidad de nulos.

Procedemos a eliminar los nulos siempre y cuando el registro completo tenga valores nulos en todas las columnas. Lo hacemos con la condicion 'how'.

In [23]:
df_games_full.dropna(inplace=True, how='all')

In [24]:
df_games_full.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32135 entries, 88310 to 120444
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   publisher       24083 non-null  object 
 1   genres          28852 non-null  object 
 2   app_name        32133 non-null  object 
 3   title           30085 non-null  object 
 4   url             32135 non-null  object 
 5   release_date    30068 non-null  object 
 6   tags            31972 non-null  object 
 7   reviews_url     32133 non-null  object 
 8   discount_price  225 non-null    float64
 9   specs           31465 non-null  object 
 10  price           30758 non-null  float64
 11  early_access    32135 non-null  float64
 12  id              32133 non-null  float64
 13  metascore       2677 non-null   object 
 14  developer       28836 non-null  object 
dtypes: float64(4), object(11)
memory usage: 3.9+ MB


Podemos ver que luego de eliminar los registros vacios, el dataset pasa de 120.445 registros a 32.135. 

Los valores nulos que aun permanecen, son registros que tienen algunas columnas con valores vacios pero aun tienen informacion valiosa para el analisis. 

Por último, lo que vamos a hacer es convertir la columna **'release_date'** a tipo *date_time*

In [25]:
# Convertimos la columna a date_time y si encuentra algun error, lo clasifica como NaT ((N)ot-(A)-(T)ime, equivalente del tiempo para NaN.)
df_games_full['release_date'] = pd.to_datetime(df_games_full['release_date'], errors='coerce')

- ### 2.2.2 **df_reviews_full**

En el dataset de 'reviews' vamos a buscar los nulos y los vamos a eliminar.

En este caso se toma esa decision porque la cantidad de nulos no representan un gran porcentaje del total de registros. 
 
En total, son 28 registros los que se eliminan. 

In [26]:
print(df_reviews_full.shape)
df_reviews_full.dropna(inplace=True)
print(df_reviews_full.shape)

(59333, 9)
(59305, 9)


Además, veremos si existen duplicados y si existen, los eliminamos.

In [27]:
df_reviews_full.duplicated(keep='first').sum()

144

In [28]:
df_reviews_full.drop_duplicates(inplace=True)

In [29]:
df_reviews_full.shape

(59161, 9)

# 3. **Feature Engineering**
***

En el planteamiento de la problematica del proyecto, se solicita la creación de la columna 'sentiment_analysis' aplicando análisis de sentimiento con NLP. La escala a tener en cuenta es la siguiente: debe tomar el valor '0' si es malo, '1' si es neutral y '2' si es positivo.

Para ello analizaremos la columna 'reviews' del dataframe 'df_reviews_full' obtenido en el paso 2.1

Utilizaremos la librería **TextBlob**.

In [30]:
from textblob import TextBlob

In [31]:
def calcular_sentimiento_textblob(texto):
    # Se calcula el sentimiento del texto
    analysis = TextBlob(texto)
    # El atributo sentiment.polarity devuelve la polaridad del sentimiento (entre -1 y 1)
    polaridad = analysis.sentiment.polarity
    return polaridad

def asignar_puntaje(polaridad):
    # Asigna 0 si la polaridad es negativa, 1 si es neutral y 2 si es positiva
    if polaridad < -0.1:
        return 0  # Negativo
    elif polaridad > 0.1:
        return 2  # Positivo
    else:
        return 1  # Neutral


In [32]:
# Creamos una copia del dataset a trabajar. 
df_sentimiento = df_reviews_full.copy()

# En el nuebo dataset, creamos dos columnas. Una de polaridad y otra de sentimiento. Se utlizan las funciones creadas en el paso anterior. 
df_sentimiento['polaridad'] = df_sentimiento['review'].astype(str).apply(calcular_sentimiento_textblob)
df_sentimiento['sentimiento'] = df_sentimiento['polaridad'].apply(asignar_puntaje)

In [33]:
df_sentimiento.head(2)

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review,polaridad,sentimiento
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,0.174444,2
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.,0.337500,2


# 4. **Desarrollo de funciones y disponibilización de datos (API)**
***

## 4.1 Primera función:** *def userdata( User_id : str )*

Debe devolver cantidad de **dinero gastado por el usuario**, el **porcentaje de recomendación en base a reviews.recommend** y **cantidad de items**.

El primer paso va a ser crear un nuevo dataframe utilizando 'merge'. Esto nos hace un join entre dataframes teninedo en cuenta una columna clave. 

El obejtivo de este nuevo dataframe es obtener la cantidad de dinero gastado por un el usuario. 

Para este caso utilizaremos como columna clave el 'item_id'. En el dataframe 'df_items_full' se encuentra bajo ese nombre, pero en el data frame 'df_games_full' se encuentra bajo el nombre 'id'.

Para poder usarlas como columna clave en el merge o join, deben ser el mismo tipo de dato. 

Procedemos a convertir las columnas a **float**. 

In [34]:
df_games_full['id'] = df_games_full['id'].astype(float)
df_items_full['item_id'] = df_items_full['item_id'].astype(float)

In [35]:
# Hacemos la union entre los dos dataframes usando como columnas claves la llamada 'item_id' e 'id'. 
df_items_games = df_items_full.merge(df_games_full, left_on='item_id', right_on='id', how='left')

In [36]:
df_items_games.head(2)

,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever,playtime_2weeks,publisher,genres,...,release_date,tags,reviews_url,discount_price,specs,price,early_access,id,metascore,developer
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,10.0,Counter-Strike,6.0,0.0,Valve,[Action],...,2000-11-01,"[Action, FPS, Multiplayer, Shooter, Classic, T...",http://steamcommunity.com/app/10/reviews/?brow...,NaN,"[Multi-player, Valve Anti-Cheat enabled]",9.99,0.0,10.0,88,Valve
1,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,20.0,Team Fortress Classic,0.0,0.0,Valve,[Action],...,1999-04-01,"[Action, FPS, Multiplayer, Classic, Shooter, C...",http://steamcommunity.com/app/20/reviews/?brow...,NaN,"[Multi-player, Valve Anti-Cheat enabled]",4.99,0.0,20.0,None,Valve


El segundo paso es crear un segundo dataframe, donde dispongamos de las columnas 'user_id', 'items_count' y 'recommend'. A partir de estas columnas podemos calcular cuántas recomendaciones tiene un usuario sobre la cantidad de ítems (juegos) que jugó. 

In [37]:
df_items_recommend = df_items_full.merge(df_reviews_full, on='user_id', how='inner')

In [46]:
df_items_recommend = df_items_recommend[['user_id', 'recommend', 'items_count']]

In [47]:
df_items_recommend

,user_id,recommend,items_count
0,76561197970982479,True,277
1,76561197970982479,True,277
2,76561197970982479,True,277
3,76561197970982479,True,277
4,76561197970982479,True,277
...,...,...,...
7782439,CaptainAmericaCw,True,7
7782440,76561198267374962,True,4
7782441,76561198267374962,True,4
7782442,76561198267374962,True,4


In [48]:
def userdata(user_id):
    '''
    La siguiente función filtra por el usuario que toma la función como input 
    y arroja el dinero gastado por dicho usuario, el porcentaje de recomendación 
    y la cantidad de items del mismo. 
    '''
    # Verificamos si el usuario existe en el DataFrame
    if user_id in df_items_games['user_id'].values and user_id in df_items_recommend['user_id'].values and user_id in df_items_full['user_id'].values:
        # Calcula la suma de la columna precio filtrando por el usuario.
        money = df_items_games[df_items_games['user_id'] == user_id]['price'].sum()
        
        # Calcula la cantidad de reviews totales de todos los usuarios en el DataFrame df_reviews_full.
        tot_recommend = df_items_recommend[df_items_recommend['user_id'] == user_id]['recommend'].sum()
       
        # Calcula la cantidad de reviews filtrando por el usuario en el DataFrame df_reviews_full.
        tot_items = df_items_recommend[df_items_recommend['user_id'] == user_id]['items_count'].iloc[0]
        
        # Calcula la cantidad de items filtrando por el usuario en el DataFrame df_items_full.
        cant_items = df_items_full[df_items_full['user_id'] == user_id]['items_count'].iloc[0]

        return {'Usuario:': user_id,
                'Cantidad de dinero gastado:': money,
                # Hacemos el cociente para calcular el porcentaje.
                'Porcentaje de recomendación:': round((tot_recommend / tot_items) * 100, 2),
                'Cantidad de items:': cant_items
                }
    else:
        return 'Usuario no encontrado'


**Ejemplo:**

In [55]:
userdata('76561197970982479')

{'Usuario:': '76561197970982479',
 'Cantidad de dinero gastado:': 3424.31,
 'Porcentaje de recomendación:': 300.0,
 'Cantidad de items:': 277}

## 4.2 Segunda función: *def countreviews( YYYY-MM-DD y YYYY-MM-DD : str )*

Cantidad de usuarios que realizaron reviews entre las fechas dadas y, el porcentaje de recomendación de los mismos en base a reviews.recommend.

El primer paso es crear un dataframe que contenga las columnas 'user_id, 'Posted', 'recommend', y 'items_count'.

Para eso haremos un merge entre el dataframe df_reviews_full y el dataframe df_items_full utilizando como columna clave el **'user_id'**

In [58]:
df_f2 = df_reviews_full.merge(df_items_full, on='user_id', how='inner')
df_f2 = df_f2[['user_id', 'posted', 'recommend', 'items_count']]
df_f2

,user_id,posted,recommend,items_count
0,76561197970982479,"Posted November 5, 2011.",True,277
1,76561197970982479,"Posted November 5, 2011.",True,277
2,76561197970982479,"Posted November 5, 2011.",True,277
3,76561197970982479,"Posted November 5, 2011.",True,277
4,76561197970982479,"Posted November 5, 2011.",True,277
...,...,...,...,...
7782439,CaptainAmericaCw,Posted July 20.,True,7
7782440,76561198267374962,Posted July 2.,True,4
7782441,76561198267374962,Posted July 2.,True,4
7782442,76561198267374962,Posted July 2.,True,4


El segundo paso es normalizar la columna 'posted' donde tenemos la fecha en formato texto. 

Eliminamos la palabra 'Posted' y convertimos la columna a tipo de dato **datetime**

In [62]:
# Eliminamos la palabra 'Posted:'
df_f2['posted']=df_f2['posted'].replace({'Posted':''},regex=True)

# Convertimos la columna a datetime.
df_f2['posted'] = pd.to_datetime(df_f2['posted'],errors='coerce')

# Eliminamos los valores con error (NaT), por ejemplo, fechas que no tenían el año.
df_f2.dropna(inplace=True)

df_f2

,user_id,posted,recommend,items_count
0,76561197970982479,2011-11-05,True,277
1,76561197970982479,2011-11-05,True,277
2,76561197970982479,2011-11-05,True,277
3,76561197970982479,2011-11-05,True,277
4,76561197970982479,2011-11-05,True,277
...,...,...,...,...
7781066,76561198232478272,2015-12-14,True,15
7781067,76561198232478272,2015-12-14,True,15
7781068,76561198232478272,2015-12-14,True,15
7781069,76561198232478272,2015-12-14,True,15


In [67]:
def countreviews(fecha_inicio, fecha_fin):
    '''
    Calcula la cantidad de usuarios que realizaron reviews entre las fechas dadas y 
    el porcentaje de recomendacion de esos usuarios. 
    '''
    # Filtra las filas del DataFrame que estén dentro del rango de fechas
    reviews_entre_fechas = df_f2[(df_f2['posted'] >= fecha_inicio) & (df_f2['posted'] <= fecha_fin)]
    
    # Calcula la cantidad de usuarios únicos que realizaron reviews en ese período
    cantidad_usuarios = reviews_entre_fechas['user_id'].count()
    
    return cantidad_usuarios
    

In [68]:
countreviews(2011-11-5,2015-12-14)

TypeError: Invalid comparison between dtype=datetime64[ns] and int

## 4.3 Tercera función: *def genre( género : str )*

Devuelve el puesto en el que se encuentra un género sobre el ranking de los mismos analizado bajo la columna PlayTimeForever.

## 4.4 Cuarta función: *def userforgenre( género : str )*

 Top 5 de usuarios con más horas de juego en el género dado, con su URL (del user) y user_id.

## 4.5 Quinta función: *def developer( desarrollador : str )*

 Cantidad de items y porcentaje de contenido Free por año según empresa desarrolladora.

## 4.1 Sexta función: *def sentiment_analysis( año : int )*

 Según el año de lanzamiento, se devuelve una lista con la cantidad de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento.

# 5. **Deployment8**
***

# 6. **Analisis exploratorio**
***

# 7. **Modelamiento**
***